<a href="https://colab.research.google.com/github/pearlielin/AICUP_2023_Fall/blob/main/aicup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive for load model checkpoint and data
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# setting

stage_1_model_path = None #'/content/drive/MyDrive/aicup_model/model_36.39.pt'
stage_2_model_path = None
stage_1_training = False
stage_2_training = False

In [ ]:
# stage 1 setting (try different pretrain model)

# plm = 'bert-base-uncased'
stage_1_plm = 'roberta-base'

stage_2_plm = 'bert-base-uncased'


In [ ]:
# The data is save in the following path

!cp -r /content/drive/MyDrive/aicup_data/* ./


In [ ]:
!pip install transformers

In [ ]:
!unzip First_Phase_ReleaseCorrection.zip
!unzip Second_Phase_Dataset.zip
!unzip opendid_test.zip

Archive:  First_Phase_ReleaseCorrection.zip
   creating: First_Phase_Release(Correction)/
  inflating: First_Phase_Release(Correction)/answer.txt  
   creating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/10.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/100.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/101.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/102.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/103.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/104.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/105.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/106.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_Dataset/107.txt  
  inflating: First_Phase_Release(Correction)/First_Phase_Text_

In [ ]:
!mkdir train_text_data
!mkdir val_text_data
!mkdir test_text_data

In [ ]:
!mv "First_Phase_Release(Correction)" First_Phase_Release

In [ ]:
!cp /content/First_Phase_Release/First_Phase_Text_Dataset/*.txt train_text_data/
!cp /content/Second_Phase_Dataset/Second_Phase_Text_Dataset/*.txt train_text_data/
!cp /content/First_Phase_Release/Validation_Release/*.txt val_text_data/

In [ ]:
!cp /content/opendid_test/*.txt test_text_data/

In [ ]:
with open('train_answer.txt', 'w') as fw:
  fw.write(open('/content/First_Phase_Release/answer.txt', 'r').read())
  fw.write(open('/content/Second_Phase_Dataset/answer.txt', 'r').read())

with open('val_answer.txt', 'w') as fw:
  fw.write(open('/content/answer.txt', 'r').read())

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
#device = torch.device('cpu')


In [ ]:

import pandas as pd
# answer data preprocess

def read_answer_df(path):
  with open(path, 'r') as f:
    answer = f.readlines()
  answer = list(map(lambda x: x.split('\t'), answer))

  columns = ['id_num', 'type', 'start_index', 'end_index', 'content', 'format_date']

  df_dict = {
      'id_num':[],
      'type':[],
      'start_index':[],
      'end_index':[],
      'content':[],
      'format_date':[]
  }

  for content in answer:
    if len(content) not in [5,6]:
      continue
    if len(content) == 6:
      df_dict['format_date'].append(content[5])
    else:
      df_dict['format_date'].append('')

    df_dict['id_num'].append(content[0])
    df_dict['type'].append(content[1])
    df_dict['start_index'].append(int(content[2]))
    df_dict['end_index'].append(int(content[3]))
    df_dict['content'].append(content[4])


  answer_df = pd.DataFrame(df_dict)
  answer_df = answer_df.set_index('id_num')
  return answer_df

In [ ]:
answer_df = read_answer_df('train_answer.txt')
val_answer_df = read_answer_df('val_answer.txt')

In [ ]:
type_list = answer_df['type'].drop_duplicates().tolist()
val_type_list = val_answer_df['type'].drop_duplicates().tolist()
type_list += val_type_list
type_list = list(set(type_list))
type_dict = {t: index + 1 for index, t in enumerate(type_list)}
type_dict['other'] = 0

In [ ]:
# force the type dict the same
type_dict = {
  'ORGANIZATION': 1, #STATE V
  'PHI': 2, # ??? PATIENT X (pos)
  'LOCATION-OTHER': 3,#COUNTRY V
  'HOSPITAL': 4,#DATE V
  'PATIENT': 5, # ORGANIZATION V
  'PHONE': 6, #LOCATION-OTHER V
  'IDNUM': 7, #ZIP V
  'ROOM': 8, # ??? ROOM X
  'DATE': 9, #HOSP V
  'STREET': 10, #TIME V
  'DURATION': 11, #DURATION V
  'DEPARTMENT': 12, #DOCTOR V
  'DOCTOR': 13, #PHONE V
  'STATE': 14, #STREET V
  'COUNTRY': 15, # ??? DEPARTMENT X (pos)
  'HOSP': 16, # ??? CITY X (pos)
  'ZIP': 17, #IDNUM V
  'SET': 18, # ??? MEDICALRECORD X (pos)
  'CITY': 19, #HOSPITAL V
  'TIME': 20, #AGE V
  'Nugal\\nHOSPITAL': 21, # ??? Nugal\nHOSPITAL X
  'AGE': 22, #PHI V
  'MEDICALRECORD': 23, #SET V
  'URL': 24, # ??? URL X
  'other': 0
}
type_list = type_list = ['ORGANIZATION', 'PHI', 'LOCATION-OTHER', 'HOSPITAL', 'PATIENT', 'PHONE', 'IDNUM', 'ROOM', 'DATE', 'STREET', 'DURATION', 'DEPARTMENT', 'DOCTOR', 'STATE', 'COUNTRY', 'HOSP', 'ZIP', 'SET', 'CITY', 'TIME', 'Nugal\\nHOSPITAL', 'AGE', 'MEDICALRECORD', 'URL']

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

'''
plm = "EleutherAI/pythia-70m" #"EleutherAI/pythia-70m-deduped"

bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")
tokenizer.padding_side = 'left'
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")
'''

'\nplm = "EleutherAI/pythia-70m" #"EleutherAI/pythia-70m-deduped"\n\nbos = \'<|endoftext|>\'\neos = \'<|END|>\'\npad = \'<|pad|>\'\nsep =\'\n\n####\n\n\'\n\nspecial_tokens_dict = {\'eos_token\': eos, \'bos_token\': bos, \'pad_token\': pad, \'sep_token\': sep}\n\ntokenizer = AutoTokenizer.from_pretrained(plm, revision="step3000")\ntokenizer.padding_side = \'left\'\nnum_added_toks = tokenizer.add_special_tokens(special_tokens_dict)\nprint(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")\n'

In [ ]:
'''
plm = 'bert-base-uncased'
pretrain_model = 'bert-base-uncased'
bos = '<|endoftext|>'
eos = '<|END|>'
pad = '<|pad|>'
sep ='\n\n####\n\n'

special_tokens_dict = {'eos_token': eos, 'bos_token': bos, 'pad_token': pad, 'sep_token': sep}

tokenizer = AutoTokenizer.from_pretrained(plm)
#tokenizer.padding_side = 'left'
#num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")
'''

'\nplm = \'bert-base-uncased\'\npretrain_model = \'bert-base-uncased\'\nbos = \'<|endoftext|>\'\neos = \'<|END|>\'\npad = \'<|pad|>\'\nsep =\'\n\n####\n\n\'\n\nspecial_tokens_dict = {\'eos_token\': eos, \'bos_token\': bos, \'pad_token\': pad, \'sep_token\': sep}\n\ntokenizer = AutoTokenizer.from_pretrained(plm)\n#tokenizer.padding_side = \'left\'\n#num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)\nprint(f"{tokenizer.pad_token}: {tokenizer.pad_token_id}")\n'

In [ ]:
# tokenizer create

stage_1_tokenizer = AutoTokenizer.from_pretrained(stage_1_plm)

stage_2_tokenizer = AutoTokenizer.from_pretrained(stage_2_plm)

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import re



def format_date_to_int_list(format_date):
  date_char_list = re.findall('\d', format_date)
  assert len(date_char_list) == 8

  date_char_list = list(map(int, date_char_list))
  output_list = []
  for digit in date_char_list:
    vec = [0] * 10
    vec[digit] = 1
    output_list += vec
  return output_list


def int_list_to_format_date(int_list):
  assert len(int_list) == 80

  output_list = []

  for i in range(8):

    vec = int_list[i*10:(i+1)*10]
    digit = np.argmax(vec)
    output_list.append(str(digit))
    if i == 3:
      output_list.append('-')
    elif i == 5:
      output_list.append('-')
    elif i == 7:
      output_list.append('\n')

  return ''.join(output_list)



In [ ]:

def format_date_to_int_list(format_date):
  date_char_list = re.findall('\d', format_date)
  if len(date_char_list) != 8:
    return [0] * 80

  date_char_list = list(map(int, date_char_list))
  output_list = []
  for digit in date_char_list:
    vec = [0] * 10
    vec[digit] = 1
    output_list += vec
  return output_list


def int_list_to_format_date(int_list):
  assert len(int_list) == 80

  output_list = []

  for i in range(8):

    vec = int_list[i*10:(i+1)*10]
    digit = np.argmax(vec)
    output_list.append(str(digit))
    if i == 3:
      output_list.append('-')
    elif i == 5:
      output_list.append('-')
    elif i == 7:
      output_list.append('\n')

  return ''.join(output_list)


In [ ]:
train_date_answer_df = answer_df
train_date_answer_df['format_date'] = train_date_answer_df.apply(lambda x: x['format_date'] if len(x['format_date']) == 11 else '', axis=1)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import math





class CasesDataset(Dataset):
  def __init__(
    self,
    case_dir_path,
    answer_df,
    split_type='test'
  ):
    self.case_dir_path = case_dir_path
    self.cases_filelist = os.listdir(case_dir_path)
    if split_type != 'test':
      self.answer_df = answer_df

    self.split_type = split_type
    self.tokenizer = stage_1_tokenizer

  def __getitem__(self, index):
    filename = self.cases_filelist[index]
    id_num = filename.replace('.txt', '')
    text = open(os.path.join(self.case_dir_path, filename)).read()
    enc_text = self.tokenizer(text, return_offsets_mapping=True, padding='max_length', truncation=True, return_tensors='pt')
    off_map = enc_text['offset_mapping'][0,:,:]
    tag_array = np.zeros(len(off_map))
    if self.split_type != 'test':
      tag = self.answer_df.loc[id_num]



      for index, row in tag.iterrows():
        si = row['start_index']
        ei = row['end_index']
        type_text = row['type']
        type_index = type_dict[type_text]
        for index, (s_i, e_i) in enumerate(off_map):
          if e_i > si and s_i < ei:
            tag_array[index] = type_index

      return id_num, enc_text, torch.tensor(tag_array).to(torch.long).view(1,-1)


    return id_num, enc_text

  def __len__(self):
    return len(self.cases_filelist)

  def collate_fn(self, batch):
    id_all = []
    y_all = []
    x_all = []

    for b in batch:
      if self.split_type != 'test':
          y_all.append(b[-1])
      id_all.append(b[0])
      x_all.append(b[1])


    for k in x_all[0]:
      x_all[0][k] = torch.cat(list(map(lambda x: x[k], x_all)))

    #del x_all[0]['offset_mapping']

    if self.split_type != 'test':
      return id_all, x_all[0], torch.cat(y_all, dim=0)
    else:
      return id_all, torch.cat(x_all)


class CasesWindowsDataset(Dataset):
  def __init__(
    self,
    case_dir_path,
    answer_df = None,
    split_type='test'
  ):
    self.case_dir_path = case_dir_path
    self.cases_filelist = os.listdir(case_dir_path)
    if split_type != 'test':
      self.answer_df = answer_df

    self.split_type = split_type
    self.tokenizer = stage_1_tokenizer
    self.windows_size = stage_1_tokenizer.model_max_length

  def __getitem__(self, index):
    filename = self.cases_filelist[index]
    id_num = filename.replace('.txt', '')
    text = open(os.path.join(self.case_dir_path, filename)).read()
    enc_text = self.tokenizer(text, return_offsets_mapping=True, padding=False, truncation=False)
    if self.split_type != 'test':
      return id_num, enc_text, self.answer_df.loc[id_num]
    return id_num, enc_text

  def __len__(self):
    return len(self.cases_filelist)

  def collate_fn(self, batch):
    id_all = []
    y_all = []
    x_all = []

    for b in batch:
      enc_text = b[1]
      token_num = len(enc_text['input_ids'])
      win_num = math.ceil(token_num / self.windows_size)

      for wid in range(win_num):
        wid_enc_text = enc_text.copy()
        for k in wid_enc_text:
          wid_enc_text[k] = wid_enc_text[k][wid*self.windows_size: (wid+1)*self.windows_size]
          if len(wid_enc_text[k]) < self.windows_size: #padding
            pad_cnt = self.windows_size - len(wid_enc_text[k])
            if k == 'input_ids':
              wid_enc_text[k] += [self.tokenizer.pad_token_id] * pad_cnt
            elif k == 'offset_mapping':
              #print(wid_enc_text[k])
              #raise Exception('stop')
              wid_enc_text[k] += [(0, 0)] * pad_cnt
            else:
              wid_enc_text[k] += [0] * pad_cnt


        id_all.append((b[0], wid))
        x_all.append(wid_enc_text)

        if self.split_type != 'test':
          off_map = wid_enc_text['offset_mapping']
          tag_array = np.zeros(len(off_map))
          tag = self.answer_df.loc[b[0]]
          for index, row in tag.iterrows():
            si = row['start_index']
            ei = row['end_index']
            type_text = row['type']
            type_index = type_dict[type_text]
            for index, (s_i, e_i) in enumerate(off_map):
              if e_i > si and s_i < ei:
                tag_array[index] = type_index
          y_all.append(tag_array)
          #torch.tensor(tag_array).to(torch.long).view(1,-1)


    for k in x_all[0]:
      if k == 'offset_mapping':
        x_all[0][k] = torch.cat(list(map(lambda x: torch.tensor(x[k]).view(1, -1, 2), x_all)))
      else:
        x_all[0][k] = torch.cat(list(map(lambda x: torch.tensor(x[k]).view(1, -1), x_all)))

    if self.split_type != 'test':
      y_all = torch.cat(list(map(lambda x: torch.tensor(x).to(torch.long).view(1, -1), y_all)))
      return id_all, x_all[0], y_all
    else:
      return id_all, x_all[0]

    #return id_all, x_all, y_all


class DateFormatDataset(Dataset):
  def __init__(
    self,
    answer_df,
    split_type='test'
  ):
    self.answer_df = answer_df[answer_df['type'] == 'DATE']
    self.split_type = split_type
    self.tokenizer = stage_2_tokenizer

  def __getitem__(self, index):
    return self.answer_df.iloc[index]

  def __len__(self):
    return len(self.answer_df)

  def collate_fn(self, batch):
    id_all = []
    y_all = []
    x_all = []

    for b in batch:
      text = b['content']
      x_all.append(text)
      id_all.append((b.name, b['start_index'], b['end_index']))
      if self.split_type != 'test':
        target_format = b['format_date']
        y_all.append(format_date_to_int_list(target_format))

    x_all = self.tokenizer(x_all, padding='max_length', truncation=True, return_tensors = 'pt')
    if self.split_type != 'test':
      y_all = torch.tensor(y_all)
      return id_all, x_all, y_all
    return id_all, x_all

train_text_path = 'train_text_data/'
val_text_path = 'val_text_data/'
test_text_path = 'test_text_data/'

#cd = CasesDataset(train_text_path, answer_df, split_type='train')
cd = CasesWindowsDataset(train_text_path, answer_df, split_type='train')
train_dataloader = DataLoader(cd, 2, collate_fn = cd.collate_fn)



val_dataset = CasesWindowsDataset(val_text_path, val_answer_df, split_type='val')
val_dataloader = DataLoader(val_dataset, 2, collate_fn = val_dataset.collate_fn)

test_dataset = CasesWindowsDataset(test_text_path, split_type='test')
test_dataloader = DataLoader(test_dataset, 2, collate_fn = test_dataset.collate_fn)


#train_date_dataset = DateFormatDataset(answer_df, split_type='train')
train_date_dataset = DateFormatDataset(train_date_answer_df, split_type='train')
train_date_dataloader = DataLoader(train_date_dataset, 2, collate_fn = train_date_dataset.collate_fn)

val_answer_df_clean = val_answer_df[val_answer_df['format_date'] != '']

val_date_dataset = DateFormatDataset(val_answer_df_clean, split_type='val')
val_date_dataloader = DataLoader(val_date_dataset, 2, collate_fn = val_date_dataset.collate_fn)



In [ ]:
from transformers import BertTokenizerFast, BatchEncoding, AutoTokenizer, AutoModelForTokenClassification, AutoModel
from torch import nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss, MSELoss



class TagModel(nn.Module):
  def __init__(self, tag2idx):
    super().__init__()
    self.tag2idx = tag2idx
    #self.bert_mul = AutoModelForTokenClassification.from_pretrained(pretrain_model, num_labels=len(type_dict))
    self.bert_mul = AutoModelForTokenClassification.from_pretrained(stage_1_plm, num_labels=len(type_dict))
  def forward(self, inputs, labels = None):
    if type(labels) != type(None):
        o = self.bert_mul(**inputs, labels = labels)
    else:
        o = self.bert_mul(**inputs)
    return o

class DateModel(nn.Module):
  def __init__(self):
    super().__init__()
    #self.bert = AutoModel.from_pretrained(pretrain_model, num_labels=len(type_dict))
    self.bert = AutoModel.from_pretrained(stage_2_plm, num_labels=len(type_dict))
    self.fc = nn.Linear(768, 80)
    self.sig = nn.Sigmoid()


  def forward(self, inputs, labels = None):
    emb = self.bert(**inputs)
    emb = emb[0][:, 0, :]
    output_logit = self.fc(emb)
    output_logit = self.sig(output_logit)
    return output_logit



model = TagModel(type_dict)
date_model = DateModel()
optimizer = Adam(model.parameters(), lr=3e-5)

date_model = DateModel()
date_optimizer = Adam(date_model.parameters(), lr=3e-5)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
#load model (if exist)

if stage_1_model_path:
  model.load_state_dict(torch.load(stage_1_model_path))

if stage_2_model_path:
  date_model.load_state_dict(torch.load(stage_2_model_path))



In [ ]:

def tag_to_type_list(idx, y_hat, offset_map):
  output_list = []

  tag_index = 0
  start_index = 0
  current_index = -1
  for b_i in range(y_hat.shape[0]):
    b = y_hat[b_i]
    ids, win_num = idx[b_i]
    for index, ti in enumerate(b):
      if ti != tag_index:
        if tag_index != 0:
          output_list.append((ids, type_list[tag_index-1], offset_map[b_i][start_index][0].item(), offset_map[b_i][index - 1][1].item())) #ids, type, si, ei
        tag_index = ti.item() # change slice
        start_index = index

  return output_list

In [ ]:
# stage 1 training (Tag Model)

from tqdm import tqdm

#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

acc_check_step = 20
epochs = 5

model.to(device)
for epoch in range(epochs):
  if stage_1_training:

    cnt = 0
    c_loss = 0
    c_acc_cnt = 0
    model.train()
    train_output_list = []
    train_tqdm_bar = tqdm(train_dataloader, desc='model training')
    for idx, x, y in train_tqdm_bar:
      x = x.to(device)
      y = y.to(device)
      offset_map = x['offset_mapping']
      del x['offset_mapping']
      o = model(x, labels = y)

      y_hat = o.logits
      y_hat = y_hat.argmax(axis=2)
      output_list = tag_to_type_list(idx, y_hat, offset_map)
      train_output_list += output_list
      loss = o.loss
      c_loss += loss.item()

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()


      cnt += len(y)
      c_acc_cnt += torch.all(torch.eq(y * x['attention_mask'], y_hat * x['attention_mask']), dim=1).int().sum() # exact match

      if cnt % acc_check_step == 0:
        train_tqdm_bar.set_description('Training acc:{:.2f}%, loss:{:.2f}'.format(c_acc_cnt / cnt * 100, c_loss / cnt))


  cnt = 0
  c_loss = 0
  c_acc_cnt = 0
  model.eval()
  val_output_list = []
  val_tqdm_bar = tqdm(val_dataloader, desc='model validation')
  with torch.no_grad():
    for idx, x, y in val_tqdm_bar:
      x = x.to(device)
      y = y.to(device)
      offset_map = x['offset_mapping']
      del x['offset_mapping']
      o = model(x, labels = y)

      y_hat = o.logits
      y_hat = y_hat.argmax(axis=2)
      output_list = tag_to_type_list(idx, y_hat, offset_map)
      val_output_list += output_list

      loss = o.loss
      c_loss += loss.item()

      cnt += len(y)
      c_acc_cnt += torch.all(torch.eq(y * x['attention_mask'], y_hat * x['attention_mask']), dim=1).int().sum() # exact match

      if cnt % acc_check_step == 0:
        val_tqdm_bar.set_description('Validation acc:{:.2f}%, loss:{:.2f}'.format(c_acc_cnt / cnt * 100, c_loss / cnt))
  if not stage_1_training:
    break


Validation acc:36.50%, loss:0.11: 100%|██████████| 280/280 [01:18<00:00,  3.55it/s]


In [ ]:
# stage 1 model saving
import torch

if stage_1_training:
  torch.save(model.state_dict(), '/content/drive/MyDrive/aicup_model/model_{:.2f}.pt'.format(c_acc_cnt / cnt * 100))


In [ ]:
# stage 1 inference
model.eval()
test_output_list = []
test_tqdm_bar = tqdm(test_dataloader, desc='model test')
with torch.no_grad():
  for idx, x in test_tqdm_bar:
    x = x.to(device)
    offset_map = x['offset_mapping']
    del x['offset_mapping']
    o = model(x)

    y_hat = o.logits
    y_hat = y_hat.argmax(axis=2)
    output_list = tag_to_type_list(idx, y_hat, offset_map)
    output_list = list(filter(lambda x: x[2] < x[3], output_list))
    test_output_list += output_list



model test: 100%|██████████| 475/475 [02:14<00:00,  3.54it/s]


In [ ]:
def output_list_to_df(output_list, raw_text_path):
  output_df = pd.DataFrame(output_list, columns=['id_num', 'type', 'start_index', 'end_index'])
  output_df['content'] = ''
  output_df['format_date'] = ''
  for i in range(len(output_df)):
    id_num = output_df.iloc[i]['id_num']
    si = int(output_df.iloc[i]['start_index'])
    ei = int(output_df.iloc[i]['end_index'])
    text = open('{}/{}.txt'.format(raw_text_path, id_num)).read()
    output_df['content'].iloc[i] = text[si:ei]
  return output_df

test_output_df = output_list_to_df(test_output_list, test_text_path)

<ipython-input-30-c4edf38c6726>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['content'].iloc[i] = text[si:ei]


In [ ]:

test_date_dataset = DateFormatDataset(test_output_df, split_type='test')
test_date_dataloader = DataLoader(test_date_dataset, 2, collate_fn = test_date_dataset.collate_fn)


In [ ]:

# stage 2 training (Date model)


from tqdm import tqdm

#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


acc_check_step = 20
epochs = 3

date_loss_fn = MSELoss()


date_model.to(device)
for epoch in range(epochs):
  if not stage_2_training:
    continue
  cnt = 0
  c_loss = 0
  c_acc_cnt = 0
  date_model.train()
  train_output_list = []
  train_tqdm_bar = tqdm(train_date_dataloader, desc='model training')
  for idx, x, y in train_tqdm_bar:
    x = x.to(device)
    y = y.to(device)

    o = date_model(x)


    y_hat = torch.zeros (o.view(-1, 8, 10).shape)
    y_hat = y_hat.to(device)
    y_hat = y_hat.scatter(2, o.view(-1, 8, 10).argmax(axis=2).unsqueeze(2), 1.0)
    y_hat = y_hat.view(-1, 80)

    output_list = list(map(int_list_to_format_date, y_hat.tolist()))
    train_output_list += output_list

    loss = date_loss_fn(o, y.float())
    c_loss += loss.item()

    loss.backward()
    date_optimizer.step()
    date_optimizer.zero_grad()

    cnt += len(y)
    c_acc_cnt += torch.all(torch.eq(y, y_hat.long()), axis=1).int().sum() # exact match

    if cnt % acc_check_step == 0:
      train_tqdm_bar.set_description('Training acc:{:.2f}%, loss:{:.2f}'.format(c_acc_cnt / cnt * 100, c_loss / cnt))

  cnt = 0
  c_loss = 0
  c_acc_cnt = 0
  date_model.eval()
  val_output_list = []
  val_tqdm_bar = tqdm(val_date_dataloader, desc='model validation')
  with torch.no_grad():
    for idx, x, y in val_tqdm_bar:
      x = x.to(device)
      y = y.to(device)
      o = date_model(x)

      y_hat = torch.zeros (o.view(-1, 8, 10).shape)
      y_hat = y_hat.to(device)
      y_hat = y_hat.scatter(2, o.view(-1, 8, 10).argmax(axis=2).unsqueeze(2), 1.0)
      y_hat = y_hat.view(-1, 80)

      output_list = list(map(int_list_to_format_date, y_hat.tolist()))
      val_output_list += output_list

      cnt += len(y)
      c_acc_cnt += torch.all(torch.eq(y, y_hat.long()), axis=1).int().sum() # exact match

      if cnt % acc_check_step == 0:
        val_tqdm_bar.set_description('Validation acc:{:.2f}%, loss:{:.2f}'.format(c_acc_cnt / cnt * 100, c_loss / cnt))




In [ ]:
# stage 2 model saving

if stage_2_training:
  torch.save(date_model.state_dict(), '/content/drive/MyDrive/aicup_model/date_model_{:.2f}.pt'.format(c_acc_cnt / cnt * 100))


In [ ]:
# stage 2 inference

model.eval()
test_output_list = []
test_tqdm_bar = tqdm(test_date_dataloader, desc='model validation')
with torch.no_grad():
  for idx, x in test_tqdm_bar:
    x = x.to(device)
    o = date_model(x)

    y_hat = torch.zeros (o.view(-1, 8, 10).shape)
    y_hat = y_hat.to(device)
    y_hat = y_hat.scatter(2, o.view(-1, 8, 10).argmax(axis=2).unsqueeze(2), 1.0)
    y_hat = y_hat.view(-1, 80)

    output_list = list(map(int_list_to_format_date, y_hat.tolist()))
    output_list = list(map(lambda x: (x[1][0], x[0]), zip(output_list, idx)))
    test_output_list += output_list


model validation: 100%|██████████| 1437/1437 [01:38<00:00, 14.66it/s]


In [ ]:

for idx, fd in test_output_list:
  test_output_df.loc[idx, 'format_date'] = fd

In [ ]:
# output final data
with open('first_stage.csv', 'w') as f:
  for i, row in test_output_df.iterrows():
    id_num = row['id_num']
    phi_name = row['type']
    start_pos = row['start_index']
    end_pos = row['end_index']
    phi_value = row['content']
    phi_value = phi_value.replace('\n', '\\n')
    phi_normalized_value = row['format_date']

    if phi_value == '':
      continue
    if id_num == '':
      continue
    if phi_name == '':
      continue

    if phi_name == 'DATE':
      seq = f"{id_num}\t{phi_name}\t{start_pos}\t{end_pos}\t{phi_value}\t{phi_normalized_value}\n"
    else:
      seq = f"{id_num}\t{phi_name}\t{start_pos}\t{end_pos}\t{phi_value}\n"
    f.write(seq)


In [ ]:
from google.colab import files
files.download('first_stage.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>